Training of Detectron2 Model. Based on Detectron2 Tutorial at:
# https://detectron2.readthedocs.io/en/latest/tutorials/getting_started.html
# https://detectron2.readthedocs.io/en/latest/

# Installation of detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

### Check Versions and import required Libraries

In [ ]:
# Check versions of detectron and torch
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup: Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
import os
import cv2
import numpy as np

### Link Colab to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
current_directory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data'

In [ ]:
# Optional Step to deregister dataset if it has already been registered 
# ( if modifying dataset while developing, can be run without any impact if dataset not registered )
DatasetCatalog.remove('fibre_Train')
DatasetCatalog.remove('fibre_Val')

### Function to load fibre images 

In [ ]:
def get_fibre_dicts(img_dir, mask_dir):
    img_files = os.listdir(img_dir) # Get list of files in the directory 
    dataset_dicts = []

    for idx, img_file in enumerate(img_files):
        record = {}

        img_path = os.path.join(img_dir, img_file)
        mask_path = os.path.join(mask_dir, f"mask{img_file[5:]}")  # Change filename from image_ to mask _ to retrieve mask

        img = cv2.imread(img_path)
        height, width = img.shape[:2]

        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        mask = cv2.imread(mask_path, 0)  # Load the mask image as grayscale

        # Get contours from mask file to calculate individual masks and bounding boxes 
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        objs = []
        for contour in contours:
            polygon = contour.squeeze().tolist()
            x, y, w, h = cv2.boundingRect(contour)

            if len(polygon) >= 6:  # Ensure at least 3 (x, y) coordinate pairs for a valid polygon - otherwise detectron will kick out error
                obj = {
                    "bbox": cv2.boundingRect(contour),
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "segmentation": [polygon],
                    "category_id": 0,
                }
                objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts

# Base directory for the Training files 
startDirectory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared'

for d in ["Train", "Val"]:
    DatasetCatalog.register("fibre_" + d, lambda d=d: get_fibre_dicts(startDirectory + '/' + d + '/images', startDirectory + '/' + d + '/masks'))
    MetadataCatalog.get("fibre_" + d).set(thing_classes=["fibre"])
fibre_metadata = MetadataCatalog.get("fibre_train")

### Check that Dataload will work properly, pick an image and show masks and bounding boxes on it 

In [ ]:
# Display examples with bounding box and masks

fibre_images = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Train/images'
fibre_masks = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Train/masks'

dataset_fibre_dicts = get_fibre_dicts(fibre_images, fibre_masks)
for d in random.sample(dataset_fibre_dicts, 1):
    print(d)  # Print File name 
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fibre_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

### Train = Fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the fibre dataset. Plot the training on a tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
cfg = get_cfg()

# cfg.OUTPUT_DIR = "./output"    
cfg.OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/" 

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("fibre_Train",)
cfg.DATASETS.TEST = ("fibre_Val")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10001    # Number of training Iterations 
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # The "RoIHead batch size". Region of interest head
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (fibre). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the fibre validation dataset. First, let's create a predictor using the model we just trained:

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

# Format of predictions such as masks and shape masks etc. can be found at 
# https://detectron2.readthedocs.io/en/latest/tutorials/models.html#model-output-format

Randomly select several samples to visualize the prediction results.

In [ ]:
dataset_dicts = get_fibre_dicts(startDirectory + '/Val/images', startDirectory + '/Val/masks')

for d in random.sample(dataset_dicts, 3):
    print(d)
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=fibre_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

Evaluate the performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#evaluator = COCOEvaluator("fibre_Val", output_dir="./output")
evaluator = COCOEvaluator("fibre_Val", output_dir=cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "fibre_Val")

# Create a predictor using the trained model
predictor = DefaultPredictor(cfg)

print(inference_on_dataset(predictor, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

In [ ]:
# Evaluate the model
eval_results = trainer.test(cfg, trainer.model)

print(eval_results)